In [ ]:
# ติดตั้งไลบรารีที่จำเป็น
%pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 23.2 MB/s eta 0:00:00
Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl (7.8 MB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------------------------ --------- 8.4/11.0 MB 47.2 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 42.9 MB/s eta 0:00:00
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   --------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: D:\Users\anyamanee\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
from athena_query_lib.query_handler import AthenaQueryHandler
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from the .env file
load_dotenv(dotenv_path=".env.aws")
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Explicitly pass credentials
handler = AthenaQueryHandler(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    aws_region=os.getenv("AWS_REGION")
)

# Load the list of table names from an external text file
file_path = 'dblist.txt'
with open(file_path, 'r') as file:
    table_names = file.read().splitlines()  # Removes the newline character
    print(table_names)

# Display available tables
print("Available tables:")
for idx, table in enumerate(table_names):
    parts = table.split("_", 2)
    print(f"{idx + 1}. {parts[2]}")

# Prompt the user to select a table
table_index = int(input(f"Please select a table by entering the corresponding number (1-{len(table_names)}): "))
selected_table_name = table_names[table_index - 1]

# Define the query
# query = f"SELECT count(*) FROM {selected_table_name};"  # No LIMIT to fetch all rows

#22 (example)
query = f"SELECT COUNT(DISTINCT ContactId) AS distinct_count FROM {selected_table_name};"  # No LIMIT to fetch all rows

# Execute the query and fetch results
print(f"Executing query for table: {selected_table_name}")
df = handler.execute_query("scbprd_pst_catdb", query)

# Show and save the data
handler.show_for_power_bi(df)
output_filename = f"{selected_table_name}_{timestamp}.csv"
handler.save_to_csv(df, output_filename)

print(f"Data saved to {output_filename}")


AWS credentials loaded successfully!
Athena query results will be saved to: s3://scbprd-data-bucket/dataquery/mkt/
['pst_api_tfpdpl_foot_fall_sensor_daily', 'pst_api_tfpdpl_foot_fall_sensor_hourly', 'pst_mongodb_tfmbk_day_building_floor_area_performance_report', 'pst_mongodb_tfmbk_day_building_floor_traffic_report_by_pin_by_hour', 'pst_mongodb_tfmbk_day_building_traffic_report_by_pin_by_hour', 'pst_mongodb_tfmbk_day_zone_area_performance_report', 'pst_mongodb_tfmbk_day_zone_traffic_report_by_pin_by_hour', 'pst_mongodb_tfpdp_day_building_floor_area_performance_report', 'pst_mongodb_tfpdp_day_building_floor_traffic_report_by_pin_by_hour', 'pst_mongodb_tfpdp_day_building_traffic_report_by_pin_by_hour', 'pst_mongodb_tfpdp_day_store_area_performance_report', 'pst_mongodb_tfpdp_day_store_traffic_report_by_pin_by_hour', 'pst_mongodb_tfpdp_day_vehicle_parking_vehicle_performance_report_by_hour', 'pst_mongodb_tfpdp_day_zone_area_performance_report', 'pst_mongodb_tfpdp_day_zone_traffic_report_by

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# โหลดข้อมูล
file_path = 'your_file_path.csv'  # เปลี่ยนเป็นเส้นทางไฟล์ของคุณ
data = pd.read_csv('Data Cross join5.csv')

# จัดกลุ่มข้อมูลตาม user_id เพื่อสร้างรายการสินค้าที่ถูกซื้อในแต่ละธุรกรรม
grouped_data = data.groupby('user_id')['item_id'].apply(list).reset_index()

# ใช้ TransactionEncoder เพื่อเข้ารหัสข้อมูลธุรกรรม
te = TransactionEncoder()
te_ary = te.fit(grouped_data['item_id']).transform(grouped_data['item_id'])
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# ทำการหาชุดรายการที่เกิดบ่อยโดยใช้ FP-Growth (ตั้งค่า min_support = 0.01)
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

# แสดง Frequent Itemsets
print(frequent_itemsets)

# หากต้องการหากฎความสัมพันธ์จากชุดรายการที่เกิดบ่อย
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# แสดง Association Rules
print(rules)

# Print the rules as a table
print(rules.to_string())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    support                         itemsets
0  0.039664        (Apple Auto Auction Data)
1  0.010802                 (Riverdale Data)
2  0.027764                  (MBK Plus Data)
3  0.880598                       (TLS Data)
4  0.054272  (Pathumwan Princess Hotel Data)
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []


In [ ]:
# Print the rules as a table
print(rules.to_string())

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Export the rules as an Excel file
output_file_path = 'association_rules_output_fpgrowth_ข้อมูล 2 ปี.xlsx'  # เปลี่ยนชื่อไฟล์ตามต้องการ
rules.to_excel(output_file_path, index=False)

print(f"Rules exported successfully to {output_file_path}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Rules exported successfully to association_rules_output_fpgrowth_ข้อมูล 6 เดือน.xlsx
